In [5]:
import numpy as np
import pandas as pd
from pandas_datareader import data as web

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline

import datetime as dt
import mplfinance as mpf

import os
from pathlib import Path

# Default Values

In [27]:
path = Path().absolute()
dataPath = path/"data"
backupPath = dataPath/"backup"

try:
    os.mkdir(dataPath)
except:
    pass

try:
    os.mkdir(backupPath)
except:
    pass


sY = 2017
sM = 1
sD = 3

start_str = "2017-01-03"
start_date = dt.datetime(sY,sM,sD)

eY = 2021
eM = 8
eD = 19

end_str = "2021-08-19"
end_date = dt.datetime(eY,eM,eD)

In [7]:
files = [x for x in os.listdir(backupPath) if os.path.isfile(os.path.join(backupPath, x))]

tickers = [os.path.splitext(x)[0] for x in files]

stock_df = pd.DataFrame(tickers, columns=["Ticker"])

stock_df

,Ticker
0,AXP
1,PRLB
2,ABEO
3,UCTT
4,JCI
...,...
2975,ADTN
2976,R
2977,GPS
2978,INFI


# Utils Functions

In [45]:
def get_ticker_path(ticker):
    return dataPath/(ticker+".csv")

def get_df_from_csv(path):
    try:
        df = pd.read_csv(path)
    except FileNotFoundError:
        print("File doesn't exists")
        return None
    else:
        return df
    
def delete_unnamed_cols(df):
    df = df.loc[:,~df.columns.str.contains('^Unnamed')]
    return df

def save_df_to_csv(df, ticker):
    df.to_csv(get_ticker_path(ticker))
    return df

def get_valid_dates(df, s_date, e_date):
    try:
        mask = (df['Date'] >= s_date) & (df['Date'] <= e_date)
        sm_df = df.loc[mask]
        sm_df = sm_df.set_index(['Date'])
        
        start_date = sm_df.index.min()
        last_date = sm_df.index.max()
        
        date_leading = '-'.join(('0' if len(x)<2 else '')+x for x in start_date.split('-'))
        date_ending = '-'.join(('0' if len(x)<2 else '')+x for x in last_date.split('-'))
        
    except Exception:
        print("Date Corrupted")
    else:
        return date_leading, date_ending
    

# Metric Functions

In [95]:
def add_daily_return_to_df(df):
    df['Daily Return'] =  (df['Adj Close']/df['Adj Close'].shift(1))-1
    return df

# Return on Investment(ROI) is the return from your initial investment
def get_roi_between_dates(df, s_date, e_date):
    
    try:
        df = df.set_index(['Date'])

        start_val = df.loc[s_date, 'Adj Close']
        end_val = df.loc[e_date, 'Adj Close']
        
        roi = (end_val - start_val)/start_val
        
        return roi
        
    except Exception:
        print("Data Corrupted")
        return None

def get_cov_between_dates(df, s_date, e_date):
    
    mask = (df['Date']>=s_date) & (df['Date']<=e_date)
    
    mean = df.loc[mask]['Adj Close'].mean()
    sd = df.loc[mask]['Adj Close'].std()

    return (sd/mean)
    
    
def get_all_cov_roi(path, tickers, s_date, e_date):
    
    cols = ['Ticker','COV','ROI']
    df = pd.DataFrame(columns = cols)
    
    for ticker in tickers:
        print("Working on:", ticker)
        
        try:
            temp_df = get_df_from_csv(backupPath/(ticker+".csv"))

            s_date,e_date = get_valid_dates(temp_df, s_date, e_date)
            cov = get_cov_between_dates(temp_df, s_date, e_date)
            roi = get_roi_between_dates(temp_df, s_date, e_date)   

            df.loc[len(df.index)] = [ticker,cov,roi]
            
        except Exception:
            pass
    
            
    return df


# Test Functions

In [99]:
ticker = tickers[0]

df = get_df_from_csv(backupPath/(ticker+".csv"))

df = add_daily_return_to_df(df)

df = delete_unnamed_cols(df)

save_df_to_csv(df, ticker)

s_date, e_date = get_valid_dates(df, start_str, end_str)

get_roi_between_dates(df, s_date, e_date)
get_cov_between_dates(df, s_date, e_date)

market_df = get_all_cov_roi(backupPath, tickers, '2019-01-01','2019-12-31')
save_df_to_csv(market_df, 'market_df')

Working on: AXP
Working on: PRLB
Working on: ABEO
Working on: UCTT
Working on: JCI
Working on: PEBK
Working on: KO
Working on: NCMI
Working on: PTC
Working on: ZEUS
Working on: WW
Working on: PWOD
Working on: TTI
Working on: LULU
Working on: L
Working on: ACNB
Working on: EYES
Working on: ILPT
Working on: THO
Working on: IPGP
Working on: EMR
Working on: BBSI
Working on: WTBA
Working on: ON
Working on: ODC
Working on: BCOV
Working on: PFSW
Working on: COF
Working on: WHG
Working on: GATX
Working on: YMAB
Working on: CSWI
Working on: AWK
Working on: GRTS
Working on: TRTX
Working on: PRMW
Working on: CZWI
Working on: XRX
Working on: NICK
Working on: CAKE
Working on: MPAA
Working on: MPW
Working on: GS
Working on: BY
Working on: NEO
Working on: CMS
Working on: PLOW
Working on: INCY
Working on: HMTV
Working on: DLB
Working on: CPLG
Working on: REGI
Working on: CURO
Working on: ULTA
Working on: APPF
Working on: AWR
Working on: CVR
Working on: MATX
Working on: HL
Working on: LPSN
Working on: 

Working on: FE
Working on: HIW
Working on: CPK
Working on: CACC
Working on: MCO
Working on: ACC
Working on: CPF
Working on: WY
Working on: FCPT
Working on: REX
Working on: AKR
Working on: STWD
Working on: ENTG
Working on: AJX
Working on: AMBC
Working on: LOPE
Working on: RCMT
Working on: NEWR
Working on: TCDA
Working on: GOOD
Working on: KEYS
Working on: ST
Working on: DPZ
Working on: MICT
Working on: JOB
Working on: AXTI
Working on: SCWX
Working on: EMKR
Working on: OAS
Working on: NTAP
Working on: EXTR
Working on: CNX
Working on: EPAM
Working on: FVE
Working on: MANH
Working on: KDP
Working on: FBNC
Working on: SRG
Working on: CLAR
Working on: MCHX
Working on: BNED
Working on: XOG
Date Corrupted
Working on: AGFS
Working on: IPG
Working on: ERIE
Working on: MGTA
Working on: MITK
Working on: IVZ
Working on: GME
Working on: SONO
Working on: VAL
Date Corrupted
Working on: OKE
Working on: SWKS
Working on: PLUS
Working on: FIS
Working on: SEM
Working on: PPL
Working on: KMT
Working on: SAL

Working on: SJW
Working on: IRBT
Working on: FWONK
Working on: HD
Working on: VHC
Working on: POST
Working on: SPR
Working on: PBPB
Working on: LEU
Working on: ALPN
Working on: INO
Working on: EXC
Working on: MLM
Working on: EEX
Working on: CNBKA
Working on: EGHT
Working on: CCK
Working on: IO
Working on: FCN
Working on: NVDA
Working on: STRL
Working on: CTIB
Working on: AIZ
Working on: HRC
Working on: SCOR
Working on: LAUR
Working on: CUBI
Working on: ZBRA
Working on: HURC
Working on: BWXT
Working on: ORLY
Working on: STAR
Working on: TGTX
Working on: ADC
Working on: MTDR
Working on: IIPR
Working on: ETSY
Working on: BHR
Working on: AYX
Working on: LBC
Working on: AVEO
Working on: EBIX
Working on: MLVF
Working on: SWN
Working on: HLT
Working on: IEX
Working on: WAT
Working on: ALSN
Working on: MTZ
Working on: REGN
Working on: PATK
Working on: CWST
Working on: NVAX
Working on: UFS
Working on: VNCE
Working on: VAPO
Working on: KEY
Working on: LSXMA
Working on: SMBC
Working on: BLMN
Work

Working on: CVLT
Working on: SAFE
Working on: AMAL
Working on: Y
Working on: NGVC
Working on: SCHW
Working on: CHEF
Working on: NNVC
Working on: CNO
Working on: RMR
Working on: MXL
Working on: GWRS
Working on: MLR
Working on: CZNC
Working on: KBH
Working on: MMC
Working on: MDB
Working on: SCSC
Working on: PFBC
Working on: HTGM
Working on: CNOB
Working on: TWST
Working on: CELC
Working on: CRVS
Working on: ATOS
Working on: LTC
Working on: CIX
Working on: DSS
Working on: LGL
Working on: BCO
Working on: MTX
Working on: ALG
Working on: RFL
Working on: VXRT
Working on: TWI
Working on: NOC
Working on: AMNB
Working on: FLO
Working on: AOSL
Working on: VIRC
Working on: CENTA
Working on: ETRN
Working on: TPR
Working on: SUM
Working on: AWI
Working on: FLIC
Working on: WVVI
Working on: AME
Working on: VCYT
Working on: ATNX
Working on: SYPR
Working on: SBH
Working on: MED
Working on: RUSHA
Working on: CBTX
Working on: DXLG
Working on: CCXI
Working on: CRVL
Working on: GLRE
Working on: JELD
Worki

Working on: NHI
Working on: VNDA
Working on: ULBI
Working on: PAYC
Working on: AMRK
Working on: MTB
Working on: SSKN
Working on: FONR
Working on: RGR
Working on: SUPN
Working on: KBAL
Working on: ADP
Working on: LCUT
Working on: BLKB
Working on: NWN
Working on: USAK
Working on: THG
Working on: CMI
Working on: XPL
Working on: KTOS
Working on: NGVT
Working on: H
Working on: AMTX
Working on: CNXN
Working on: THFF
Working on: CCI
Working on: MEIP
Working on: NWBI
Working on: ALLY
Working on: PFMT
Working on: PKI
Working on: FLXN
Working on: TMO
Working on: TTD
Working on: GNK
Working on: RIG
Working on: ANIX
Working on: ISBC
Working on: KAMN
Working on: NWPX
Working on: ESNT
Working on: KAR
Working on: CRAI
Working on: HOLX
Working on: PFIN
Working on: RRTS
Working on: TRMB
Working on: IOVA
Working on: ZNGA
Working on: FISV
Working on: BCBP
Working on: CDMO
Working on: AXAS
Working on: HTLD
Working on: GPI
Working on: TRMK
Working on: AUTO
Working on: NNBR
Working on: MODN
Working on: TDC


Working on: CIEN
Working on: CPE
Working on: WINA
Working on: ARES
Working on: VERU
Working on: PLCE
Working on: CGNX
Working on: ELSE
Working on: OESX
Working on: BMI
Working on: CECE
Working on: RLGY
Working on: EIX
Working on: CTSO
Working on: OXY
Working on: ETR
Working on: RAIL
Working on: ENOB
Working on: ZM
Working on: OSUR
Working on: PANL
Working on: ACHV
Working on: RRGB
Working on: IRTC
Working on: ADMS
Working on: FTDR
Working on: ROL
Working on: INVH
Working on: TR
Working on: SGRP
Working on: MBWM
Working on: FMAO
Working on: BJRI
Working on: OFG
Working on: PNRG
Working on: AIRG
Working on: CHMG
Working on: RBBN
Working on: ARDX
Working on: SND
Working on: BKD
Working on: OCX
Working on: PRO
Working on: NMRK
Working on: LIN
Working on: NSEC
Working on: DELL
Working on: FARO
Working on: CMCO
Working on: CCF
Working on: SVBI
Working on: CHMA
Working on: TILE
Working on: EVH
Working on: DCI
Working on: BCRX
Working on: ALNY
Working on: NKSH
Working on: HES
Working on: PAR
W

,Ticker,COV,ROI
0,AXP,0.069714,0.320188
1,PRLB,0.070337,-0.089972
2,ABEO,0.448976,-0.534851
3,UCTT,0.282821,1.703917
4,JCI,0.096233,0.369792
...,...,...,...
2965,ADTN,0.032191,0.046376
2966,R,0.020077,0.041530
2967,GPS,0.037099,0.007982
2968,INFI,0.100069,-0.111111


In [98]:
market_df.sort_values(by=['ROI'], ascending=False).head(20)

,Ticker,COV,ROI
1933,FCEL,0.506757,8.296296
124,DRRX,0.567597,6.037037
1636,SVRA,0.625021,3.497992
1556,CCXI,0.518287,3.225427
2295,AXSM,0.559061,3.096710
555,CDLX,0.356442,2.706368
2153,ITCI,0.656815,2.570240
398,ARWR,0.400710,2.496693
2851,KOD,0.453787,2.314141
92,NVRO,0.299194,2.215869
